### Python Notebook for reducing raw data from SLAC MeV UED

In [ ]:
%load_ext autoreload
%autoreload 2
# Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sys

sys.path.append('/cds/group/ued/scratch/jupyter_notebook_UED_solid_sample_codes/python_packages/')

from ued_gas_phase_FY18_utilities import *

from datetime import datetime
from scipy.ndimage import median_filter 
import h5py


import tqdm
from tqdm import tnrange

from skimage.filters import threshold_otsu, threshold_local
import cv2

from scipy.interpolate import interp2d, griddata
from copy import deepcopy

from scipy.optimize import fsolve

from scipy.special import erfcx
import scipy.optimize as opt

from mpl_toolkits import mplot3d

from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import disk, rectangle
from skimage.transform import AffineTransform, warp

from skimage.registration import phase_cross_correlation

In [ ]:
from scipy import ndimage

In [ ]:
# plotting parameters
default_text_size=15
font_times = {'family':'Arial'}

plt.rc('font', size=default_text_size)          # controls default text sizes
plt.rc('axes', titlesize=default_text_size)     # fontsize of the axes title
plt.rc('axes', labelsize=default_text_size)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=default_text_size)    # fontsize of the tick labels
plt.rc('ytick', labelsize=default_text_size)    # fontsize of the tick labels
plt.rc('figure', titlesize=default_text_size)  # fontsize of the figure title
plt.rc('font', **font_times)
plt.rc('figure', figsize=(8,6))
plt.rc('legend', fontsize=15, frameon=False)
plt.rc('lines', linewidth=2)
lw = 2 #default line width
ms = 15
plt.rcParams.update({'figure.max_open_warning': 0})

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Read in data 

In [ ]:
path_df = '/cds/group/ued/data/ueduu8503/screening/20211005/Run/20211005_1759/'

In [ ]:
# Image dimensions
num_row = 1024
num_col = 1024

file_list_df = get_diffraction_file_list(path_df, keyword='ANDOR1*tif')

# file_list_df = file_list_df[0:100]

num_image_df = len(file_list_df)

print('Number of diffraction images: %d'%(num_image_df))
img_set = assign_shared_memory((num_image_df, num_row, num_col))
scan_num = assign_shared_memory((num_image_df,))
delay =  assign_shared_memory((num_image_df,))

# Read diffraction data
read_diffraction_data_sharedarray(file_list_df, img_set, scan_num, delay)


## subtract camera background 

In [ ]:
width_1 = 30
width_2 = 15
bg_slice_1 = np.s_[0:width_1, 0:width_1]
bg_slice_2 = np.s_[1024-width_1:1024, 0:width_1]
bg_slice_3 = np.s_[0:width_2, 1024-width_2:1024]
bg_slice_4 = np.s_[1024-width_2:1024, 1024-width_2:1024]

bg_slice_set = [bg_slice_1, bg_slice_2, bg_slice_3, bg_slice_4]

tttt = np.ones(img_set[0,:,:].shape)
tttt[bg_slice_1] = np.nan
tttt[bg_slice_2] = np.nan
tttt[bg_slice_3] = np.nan
tttt[bg_slice_4] = np.nan

img_temp = img_set[34,:,:]
plt.figure()
for ind, bg in enumerate(bg_slice_set):
    ct, bins = np.histogram(np.ndarray.flatten(img_temp[bg]), bins=20, normed=True)
    plt.plot(bins[1:], ct, label='Bg slice %d'%(ind+1))
plt.legend()
plt.xlabel('Intensity (arb. u.)')
plt.ylabel('Frequency (arb. u.)')

    
plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.imshow(img_temp[bg_slice_1])
plt.clim([400, 1000])
plt.title('Bg slice 1')

plt.subplot(2,2,2)
plt.imshow(img_temp[bg_slice_2])
plt.clim([400, 1000])
plt.title('Bg slice 2')

plt.subplot(2,2,3)
plt.imshow(img_temp[bg_slice_3])
plt.clim([400, 1000])
plt.title('Bg slice 3')

plt.subplot(2,2,4)
plt.imshow(img_temp[bg_slice_4])
plt.clim([400, 1000])
plt.title('Bg slice 4')

plt.figure(figsize=(10, 5))
plt.imshow(img_set[0,:,:]*tttt)
plt.clim([600, 1000])
plt.colorbar()


### center mask 

In [ ]:
ndimage.measurements.center_of_mass(img_temp)

In [ ]:
cm_x = 519 #center of mass x and y 
cm_y = 523
# cm = 1-gen_circular_mask(cm_x, cm_y, 187, num_col, num_row)
mask_hole =  1-gen_circular_mask(cm_x, cm_y, 46, 1024, 1024)
mask_hole[mask_hole<1] = np.nan

plt.figure()
plt.subplot(1,2,1)
plt.imshow(np.log(img_set[0,:,:]))
plt.clim([4, 9])
plt.title('Raw')
plt.subplot(1,2,2)
plt.imshow(np.log(img_set[0,:,:])*mask_hole)
# plt.colorbar()
plt.clim([4, 9])
plt.title('With mask')

In [ ]:
bg_set = np.zeros((num_image_df,1))
inten_set = np.zeros((num_image_df,1))
for ind_img in tnrange(num_image_df):
    img_temp = img_set[ind_img,:,:]
    temp = np.concatenate((img_temp[bg_slice_1].flatten(), img_temp[bg_slice_2].flatten()))
    bg_set[ind_img] = np.median(temp)
    img_set[ind_img,:,:] = img_set[ind_img,:,:] - bg_set[ind_img]
    inten_set[ind_img] = np.nansum(img_set[ind_img,:,:]*mask_hole)



In [ ]:
plt.figure(figsize=(8,6))
plt.plot(bg_set, 'r-o')
plt.xlabel('Image #')
plt.ylabel('Median count within bg windwon (arb. u.)')

plt.figure(figsize=(8,6))
plt.plot(inten_set, 'r-', label=f'rms={np.std(inten_set)/np.mean(inten_set)*100:.2f}%')
plt.xlabel('Image #')
plt.ylabel('Total intensity (arb. u.)')
plt.legend()


## Align images 

#### Affine Transform :
skimage.transform.AffineTransform \
form: \
X = a0 * x + a1 * y + a2 = sx * x * cos(rotation) - sy * y * sin(rotation + shear) + a2 \
Y = b0 * x + b1 * y + b2 = sx * x * cos(rotation) - sy * y * sin(rotation + shear) + b2 \
sx and sy are scale factors in x & y directions \ 
homogenous tranformation matrix is: [[a0 a1 a2]
[b0 b1 b2] [0 0 1]]

### Phase cross-correlation 
Used to identify the relative shift between two similar-sized images \
Obtains an initial estimate of the cross-correlation peak by an FFT and then refines the shift estimation by upsampling the DFT only in a small neighborhood of that estimate by means of a matrix-multiply DFT

params: **reference image**, **moving image (image to register)**, **upsample factor**: (int, optional) images will be registered to within 1 / upsample_factor of a pixel 
upsample factor = 100 means images will be registered with 1/100th of a pixel, **space**='real' or 'fourier' real: data will be FFT'd to compute the correlation 

returns: **shifts**: (ndarray) shift vector (in pixels) required to register moving image with reference image, **error** translation invariant normalized RMS error between ref image and moving image, **phasediff** global phase difference between 2 images

### Warp
Warp an image according to a given coord transformation
params: **image**, **inverse_map** a transofrmation object, callable cr = f(cr, ** kwargs ** ), ndarray; inverse coordinate map, which transforms coords in teh output images into corresponding coords in the input image **mode**: 'wrap': pad an array with the wrap of the vector along the axis. the first values are used to pad the end and the end values are used to pad the beginning 

In [ ]:
from scipy.ndimage import median_filter
from skimage.transform import AffineTransform, warp

index_slice_row = slice(470, 530) # choosing the two points to look at 
index_slice_col = slice(290, 360) # choosing the two points to look at

plt.figure()
plt.subplot(1,2,1)
plt.imshow(np.log(img_set[0]))
plt.title('Full image')
plt.subplot(1,2,2)
plt.imshow(np.log(img_set[0,index_slice_row, index_slice_col]))
plt.title('ROI for cross-correlation')

filter_size = [9, 9]
img_temp0 = median_filter(img_set[0,index_slice_row, index_slice_col], filter_size) # looking at these points again
img_temp = median_filter(img_set[10,index_slice_row, index_slice_col], filter_size) # 9x9 region, first and 10th image

plt.figure()
plt.subplot(1,2,1)
plt.imshow(img_temp0)
plt.subplot(1,2,2)
plt.imshow(img_temp)

In [ ]:
shift_set = []
img_set_aligned = []
img_set_aligned.append(img_set[0,:,:])

# for ind in tnrange(1):
for ind in tnrange(img_set.shape[0]-1): #range one less than full set, we are aligning to the first image 
    img_temp = median_filter(img_set[ind+1, index_slice_row, index_slice_col], filter_size)
    shift_filtered, error, diffphase = phase_cross_correlation(img_temp, img_temp0, upsample_factor=100, space='real')
    shift_set.append(shift_filtered)
    transform = AffineTransform(translation=shift_filtered[::-1]) #reversing the order 
    img_set_aligned.append( warp(img_set[ind+1,:,:], transform, mode='wrap', preserve_range=True) )

shift_set = np.array(shift_set)

In [ ]:
img_set_aligned = np.array(img_set_aligned)

In [ ]:
plt.figure()
plt.imshow(np.log(np.mean(img_set_aligned, axis=0)))
plt.title('Shift corrected')

plt.figure()
plt.imshow(np.log(np.mean(img_set, axis=0)))
plt.title('Raw')

plt.figure()
plt.imshow((np.mean(img_set, axis=0)-np.mean(img_set_aligned, axis=0)))
plt.title('Raw-shift_corrected')
plt.clim([-100, 100])


plt.figure()
plt.plot(shift_set[:,0], shift_set[:,1], 'o')
plt.xlabel('Shift y (px)')
plt.ylabel('Shift x (px)')

plt.figure()
plt.plot(shift_set[:,1], label='shift x')
plt.plot(shift_set[:,0], label='shift y')
plt.ylabel('Shift (px)')
plt.legend()

In [ ]:
path_df

In [ ]:
#import h5py 
hf = h5py.File('/cds/group/ued/scratch/ueduu8503/reduced_data/ued8503_20211005_1759.h5', 'w') 
hf.create_dataset('Data_description', data='Time scan of sample E1 MoS2/WS2_4.8deg 660nm pump 200um e-beam size ~2mJ/cm2 RT')
hf.create_dataset('Folder', data=path_df)
hf.create_dataset('scan_num', data=scan_num)
hf.create_dataset('delay', data=delay)
hf.create_dataset('img_set_aligned', data=img_set_aligned)
hf.create_dataset('shift_set', data=shift_set)
hf.create_dataset('inten_set', data=inten_set)
#hf.create_dataset('index_slice_col', data=index_slice_col) #gives error
#hf.create_dataset('index_slice_row', data=index_slice_row) #gives error
hf.create_dataset('filter_size', data=filter_size)
#hf.create_dataset('bg_slice_set', data=bg_slice_set) #gives error 
hf.create_dataset('bg_set', data=bg_set)
hf.close()